## **1. DESKRIPSI DATABASE**

### **1.1 RINGKASAN ISI DATABASE DAN HARAPAN OUTPUT DARI ANALISIS**

NORTHWIND_DATABASE merupakan database dari sebuah perusahaan yang bergerak dalam bidang F&B (food and beverage). Database ini berisi hampir semua data operasional perusahaan, meliputi informasi mengenai detail pekerja, penjualan, pelanggan, supplier, sampai dengan partner jasa ekspedisi.

Untuk mengoptimalkan kinerja pekerja dalam penjualan produk, perusahaan ingin mengambil insight dari database yang mereka miliki.

### **1.2 INFORMASI DATABASE**
sumber database: https://drive.google.com/drive/folders/1fTHrwh_gcLsOFKXHnUzUGEu_APxLoD9i

Database memiliki total 13 tabel yang terhubung baik secara langsung ataupun tidak langsung. Berikut adalah ulasan singkat mengenai isi tabel-tabel tersebut:
1. Employees            : Berisi data mengenai detail pekerja.
2. EmployeeTerritories  : Berisi data mengenai cakupan wilayah kerja setiap pekerja.
3. Territories          : Berisi data mengenai deskripsi wilayah kerja.
4. Region               : Berisi data mengenai deksripsi region dari setiap wilayah kerja.
5. Orders               : Berisi data mengenai informasi umum order, seperti ID order, ID customer, ID employee, tanggal order, tanggal penggiriman, jasa ekspedisi, alamat pengiriman, dsb.
6. Shippers             : Berisi data mengenai detail dari jasa ekspedisi yang dipilih di tabel Orders.
7. OrderDetails         : Berisi data dari orderan customer mengenai ID produk, harga satuan, qty, serta diskon jika ada.
8. Products             : Berisi data mengenai produk yang di order, seperti nama produk, kategori, supplier, stok, harga satuan, dsb.
9. Categories           : Berisi data mengenai deskripsi dari kategori produk.
10. Supplier            : Berisi data mengenai detail supplier.
11. Customers           : Berisi data mengenai detail pelanggan.
12. CustomerCustomerDemo: - (tidak ada data tersedia)
13. CustomerDemographic : - (tidak ada data tersedia)

### **1.3 QUESTION LIST**
1. Faktor apa dari employee yang berpengaruh terhadap total profit perusahaan?
2. Apa yang harus dilakukan ke employee agar profit perusahaan meningkat?

## **2. DATABASE**

### **2.1 CONNECTING TO DATABASE**

Pada tahap ini, kita akan membuat koneksi ke database dengan menggunakan module mysql.connector. Database yang akan digunakan adalah "northwind_database".

In [176]:
# import module yang diperlukan

import mysql.connector as sqlcon # module yang digunakan untuk dapat membuat koneksi dengan database
import numpy as np
import pandas as pd

In [177]:
# Connect to database

db = sqlcon.connect(
    host= 'localhost',
    user= 'root',
    password= '123456',
    database= 'northwind_database'
)

### **2.2 CREATE FUNCTION TO QUERY AND SAVE AS DATAFRAME**

Setelah koneksi ke database berhasil dilakukan, langkah selanjutnya ada mengambil data dari database tersebut. Namun, agar proses pengambilan data lebih muda dan bentuk outputnya menjadi seperti yang kita inginkan (dataframe), kita buat terlebih dahulu sebuah fungsi. Fungsi ini berfungsi untuk melakukan query dan menyimpan hasilnya dalam bentuk dataframe.

In [178]:
# membuat akses ke database
curs = db.cursor()

# fungsi untuk melakukan query dan menyimpan hasilnya sebagai dataframe
def df(query):
    curs.execute(query) # eksekusi query
    result = curs.fetchall() # menyimpan hasil query dalam variabel "result"
    df = pd.DataFrame(result, columns= curs.column_names) # membuat dataframe yang disimpan dalam variabel "df" dari hasil query dalam variabel "result"
    return df # mengembalikan variabel "df" sebagai output fungsi

### **2.3 MENGAMBIL DATA YANG NANTINYA AKAN MENJADI BAHAN ANALISA**

Untuk keperluan analisa ini, terdapat beberapa jenis data yang akan diambil dari beberapa tabel dalam database. Berikut adalah detailnya:
1. EmplyeedID, dari tabel employees
2. LastName, dari tabel employees
3. FirstName, dari tabel employees
4. Title, dari tabel employees
5. TitleOfCourtesy, dari tabel employees
6. HireDate, dari tabel employees
7. Country, dari tabel employees
8. ReportTo, dari tabel employees
9. Salary, dari tabel employees
10. TerritoryDescription, dari tabel territories
11. RegionDescription, dari tabel region
12. OrderID, dari tabel orders
13. CustomerID, dari tabel orders
14. OrderDate, dari tabel orders
15. RequiredDate, dari tabel orders
16. ShippedDate, dari tabel orders
17. ShipCity, dari tabel orders
18. ShipRegion, dari tabel orders
19. ShipCountry, dari tabel orders
20. ProductID, dari tabel orderdetails
21. UnitPrice dari tabel orderdetails
22. Quantity, dari tabel orderdetails
23. Discount, dari tabel orderdetails
24. UnitPrice, dari tabel products
25. CategoryName, dari tabel categories

Tambahan:
1. Name, dari hasil operasi LastName + FirstName dari tabel employees
2. profit, dari hasil operasi UnitPriceSell - UnitPriceBuy
3. HandlingTime, dari hasil operasi ShippedDate - OrderDate
4. OTD (on time delivery), dari hasil operasi RequiredDate - ShippedDate


In [179]:
# Menjalankan fungsi "df" dengan inputan query

# simpan hasil query dalam variabel df1
df1 = df('''
SELECT 
R.RegionDescription AS EmployeeRegion,
T.TerritoryDescription AS EmployeeTerritory,
E.EmployeeID,
E.LastName,
E.FirstName,
E.Title,
E.TitleOfCourtesy,
E.HireDate,
E.Country,
E.ReportsTo,
E.Salary,
O.OrderID,
O.CustomerID,
O.OrderDate,
O.RequiredDate,
O.ShippedDate,
O.ShipCity,
O.ShipRegion,
O.ShipCountry,
OD.ProductID,
OD.UnitPrice AS UnitPriceBuy,
OD.Quantity,
OD.Discount,
P.UnitPrice AS UnitPriceSell,
C.CategoryName
FROM region R RIGHT JOIN
territories T ON R.RegionID = T.RegionID RIGHT JOIN
employeeterritories ET ON T.TerritoryID = ET.TerritoryID RIGHT JOIN
employees E ON ET.EmployeeID = E.EmployeeID LEFT JOIN
orders O ON E.EmployeeID = O.EmployeeID LEFT JOIN
orderdetails OD ON O.OrderID = OD.OrderID LEFT JOIN
products P ON OD.ProductID = P.ProductID LEFT JOIN
categories C ON P.CategoryID = C.CategoryID;
''')

df1.head()

,EmployeeRegion,EmployeeTerritory,EmployeeID,LastName,FirstName,Title,TitleOfCourtesy,HireDate,Country,ReportsTo,...,ShippedDate,ShipCity,ShipRegion,ShipCountry,ProductID,UnitPriceBuy,Quantity,Discount,UnitPriceSell,CategoryName
0,Eastern ...,Wilton ...,1,Davolio,Nancy,Sales Representative,Ms.,1992-05-01,USA,2.0,...,1996-07-23,Graz,None,Austria,2,15.2000,50,0.0,19.0000,Beverages
1,Eastern ...,Wilton ...,1,Davolio,Nancy,Sales Representative,Ms.,1992-05-01,USA,2.0,...,1996-07-23,Graz,None,Austria,5,17.0000,65,0.0,21.3500,Condiments
2,Eastern ...,Wilton ...,1,Davolio,Nancy,Sales Representative,Ms.,1992-05-01,USA,2.0,...,1996-07-23,Graz,None,Austria,32,25.6000,6,0.0,32.0000,Dairy Products
3,Eastern ...,Wilton ...,1,Davolio,Nancy,Sales Representative,Ms.,1992-05-01,USA,2.0,...,1996-08-02,Oulu,None,Finland,36,15.2000,30,0.0,19.0000,Seafood
4,Eastern ...,Wilton ...,1,Davolio,Nancy,Sales Representative,Ms.,1992-05-01,USA,2.0,...,1996-08-02,Oulu,None,Finland,43,36.8000,25,0.0,46.0000,Beverages


## **3. DATA MANIPULATION**

Sebelum melakukan analisa terhadap data, data perlu untuk dicek dari keberadaan anomali. Anomali data dapat dalam bentuk salah tipe data, missing value dan data kembar atau duplikat.

Selain memastikan bahwa data telah bersih, kita juga dapat mengecek apakah ada tambahan informasi yang dapat diekstrak dari data yang ada sehingga didapatkan insight yang dalam terhadap data. Untuk ini kita bisa membuat feature atau kolom baru dari kolom yang sudah ada dalam dataframe.

### **3.1 DATA ANOMALIES**

#### **3.1.1 MENGECEK DATA ANOMALIES DARI METHOD "INFO"**

In [180]:
# menggunakan method "info" untuk melihat infromasi umum dataframe

df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10129 entries, 0 to 10128
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   EmployeeRegion     10129 non-null  object        
 1   EmployeeTerritory  10129 non-null  object        
 2   EmployeeID         10129 non-null  int64         
 3   LastName           10129 non-null  object        
 4   FirstName          10129 non-null  object        
 5   Title              10129 non-null  object        
 6   TitleOfCourtesy    10129 non-null  object        
 7   HireDate           10129 non-null  datetime64[ns]
 8   Country            10129 non-null  object        
 9   ReportsTo          8442 non-null   float64       
 10  Salary             10129 non-null  float64       
 11  OrderID            10129 non-null  int64         
 12  CustomerID         10129 non-null  object        
 13  OrderDate          10129 non-null  datetime64[ns]
 14  Requir

Dari informasi dua umum diatas, terdapat beberapa poin yang patut untuk di garis bawahi:
1. Tipe data:
    - "UnitPriceSell" berisi data numerik yang mana seharusnya memiliki tipe data FLOAT bukan OBJECT.
    - "UnitPriceBuy" berisi data numerk yang mana seharusnya memiliki tipe data FLOAT bukan OBJECT.
2. Missing value:
    - "ReportTo" memiliki missing value sekitar 17% dari total data. Namun perlu dicatat bahwa ada employee yang memang tidak memiliki atasan/ tidak memiliki nilai "ReportTo". Jadi, kita harus memastikan datang dari employee dengan "EmployeeID" berapa missing value berasal.
    - "ShippedDate" memiliki missing value sekitar 3% dari total data. "ShippedDate" dalam database ini bisa diinterpretasikan sebagai status dari order dalam database ini karena tidak ada data "status_order" secara spesifik. Artinya, apabila "ShippedDate" terisi order dapat dianggap CLOSED. Sementara itu apabilai "ShippedDate" kosong, dapat dianggap bahwa order masih OPEN atau tidak terselesaikan. Jadi, untuk missing value pada kolom "ShippedDate" tidak akan ditangani dan akan menjadi bahan analisa.
    - "ShipRegion" memiliki nilai missing value sekitar 62% dari total data. Mempertimbangkan presentase yang tinggi dari missing value di kolom tersebut dan adanya data "ShipCity" dan "ShipCountry" yang cukup sebagai bahan analisa tempat tujuan/ asal customer, kolom "ShipRegion" akan dihapus dari tabel analisa.

#### **3.1.2 MENANGGULANGI ANOMALI SESUAI DENGAN HASIL ANALISA METHOD "INFO"**

##### **3.1.2.1 MERUBAH TIPE DATA YANG SALAH**

##### **3.1.2.1.1 Merubah Tipe Data Kolom "UnitPriceSell"**

In [181]:
# Merubah tipe data kolom "UnitPriceSell" menjadi FLOAT
df1['UnitPriceSell'] = pd.to_numeric(df1['UnitPriceSell'])

# mengecek kembali tipe data kolom "UnitPriceSell"
df1['UnitPriceSell'].dtype


dtype('float64')

##### **3.1.2.1.1 Merubah Tipe Data Kolom "UnitPriceBuy"**

In [182]:
# Merubah tipe data kolom "UnitPriceBuy" menjadi FLOAT
df1['UnitPriceBuy'] = pd.to_numeric(df1['UnitPriceBuy'])

# mengecek kembali tipe data kolom "UnitPriceBuy"
df1['UnitPriceBuy'].dtype

dtype('float64')

##### **3.1.2.2 MENGATASI MISSING VALUE**

##### **3.1.2.2.1 Missing Value Pada Kolom "ReportsTo"**

In [183]:
# Mengecek EmployeeID dari data dengan ReportsTo berisi missing value
df1[df1['ReportsTo'].isnull() == True]['EmployeeID'].groupby(df1['EmployeeID']).count()

EmployeeID
2    1687
Name: EmployeeID, dtype: int64

Jumlah data kosong pada kolom "reportsTo" = 10129 - 8442 = 1687.
Jadi terbukti bahwa missing value pada kolom "ReportsTo" berasal hanya dari EmployeeID = 2 yang mana memang tidak memiliki atasan. Oleh karena itu, kolom "ReportsTo" tidak diisi/ kosong dan kita akan mengabaikan missing value pada kolom "ReportsTo".

##### **3.1.2.2.2 Missing Value Pada Kolom "ShipRegion"**

In [184]:
# Menghapus kolom "ShipRegion" yang memiliki missing value 62%
df1.drop('ShipRegion', axis=1, inplace=True)

In [185]:
# Mengecek kembali method "info" untuk memastikan kolom "ShipRegion" telah terhapus
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10129 entries, 0 to 10128
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   EmployeeRegion     10129 non-null  object        
 1   EmployeeTerritory  10129 non-null  object        
 2   EmployeeID         10129 non-null  int64         
 3   LastName           10129 non-null  object        
 4   FirstName          10129 non-null  object        
 5   Title              10129 non-null  object        
 6   TitleOfCourtesy    10129 non-null  object        
 7   HireDate           10129 non-null  datetime64[ns]
 8   Country            10129 non-null  object        
 9   ReportsTo          8442 non-null   float64       
 10  Salary             10129 non-null  float64       
 11  OrderID            10129 non-null  int64         
 12  CustomerID         10129 non-null  object        
 13  OrderDate          10129 non-null  datetime64[ns]
 14  Requir

#### **3.1.3 MENGECEK DATA DUPLIKAT**

In [186]:
df1[df1.duplicated()][['EmployeeID', 'OrderID', 'OrderDate', 'RequiredDate', 'CustomerID', 'ShipCity', 'ProductID', 'Quantity']]

,EmployeeID,OrderID,OrderDate,RequiredDate,CustomerID,ShipCity,ProductID,Quantity
5389,5,10248,1996-07-04,1996-08-01,VINET,Reims,11,12
5390,5,10248,1996-07-04,1996-08-01,VINET,Reims,42,10
5391,5,10248,1996-07-04,1996-08-01,VINET,Reims,72,5
5392,5,10254,1996-07-11,1996-08-08,CHOPS,Bern,24,15
5393,5,10254,1996-07-11,1996-08-08,CHOPS,Bern,55,21
...,...,...,...,...,...,...,...,...
5501,5,10954,1998-03-17,1998-04-28,LINOD,I. de Margarita,16,28
5502,5,10954,1998-03-17,1998-04-28,LINOD,I. de Margarita,31,25
5503,5,10954,1998-03-17,1998-04-28,LINOD,I. de Margarita,45,30
5504,5,10954,1998-03-17,1998-04-28,LINOD,I. de Margarita,60,24


Dari tabel diatas, diketahui terdapat 117 baris yang memiliki data duplikat.

In [187]:
# melihat data yang duplikat pada salah satu contoh "OrderID"=10248
df1[df1['OrderID'] == 10248][['EmployeeID', 'OrderID', 'OrderDate', 'CustomerID', 'ShipCity', 'ProductID', 'Quantity']]

,EmployeeID,OrderID,OrderDate,CustomerID,ShipCity,ProductID,Quantity
4921,5,10248,1996-07-04,VINET,Reims,11,12
4922,5,10248,1996-07-04,VINET,Reims,42,10
4923,5,10248,1996-07-04,VINET,Reims,72,5
5038,5,10248,1996-07-04,VINET,Reims,11,12
5039,5,10248,1996-07-04,VINET,Reims,42,10
5040,5,10248,1996-07-04,VINET,Reims,72,5
5155,5,10248,1996-07-04,VINET,Reims,11,12
5156,5,10248,1996-07-04,VINET,Reims,42,10
5157,5,10248,1996-07-04,VINET,Reims,72,5
5272,5,10248,1996-07-04,VINET,Reims,11,12


Dari tabel diatas, Terlihat bahwa memang terdapat beberapa duplikat, seperti antara nomor index 4921, 5038, 5155, dsb.

Karena jumlahnya yang hanya sekitar 1.1% dari total data dan potensi bias pada saat analisis dengan adanya data duplikat, maka kita hapus baris dengan data duplikat.

In [188]:
# Menghapus baris yang memiliki data duplikat
df1.drop_duplicates(inplace=True)

In [189]:
# mengecek kembali kemungkinan adanya data duplikat
df1[df1.duplicated()]

,EmployeeRegion,EmployeeTerritory,EmployeeID,LastName,FirstName,Title,TitleOfCourtesy,HireDate,Country,ReportsTo,...,RequiredDate,ShippedDate,ShipCity,ShipCountry,ProductID,UnitPriceBuy,Quantity,Discount,UnitPriceSell,CategoryName


Dari tabel diatas, diketahui bahwa data duplikat sudah tidak ada.

### **3.2 MEMBUAT FITUR/ KOLOM BARU SEBAGAI TAMBAHAN INSIGHT DARI DATA YANG ADA**

#### **3.2.1 Membuat Kolom/ Fitur Baru "Name" Sebagai Pengganti Kolom "FirtsName" dan "LastName"**

Membuat kolom baru "Name" sebagai hasil penggabungan kolom "FirstName" dan "LastName" untuk menggantikan kedua kolom tersebut. Hal ini dilakukan untuk menyederhakan data dari 2 kolom menjadi hanya satu kolom.

##### **3.2.1.1 Membuat Kolom/ Fitur baru "Name"**

In [190]:
# Membuat kolom baru "Name" sebagai hasil penggabungan kolom "FirstName" dan "LastName".

df1.insert(3, 'Name', df1['FirstName'] + ' ' +df1['LastName'])

##### **3.2.1.2 Menghapus Kolom/ Fitur baru "FirtsName" dan "LastName**

In [191]:
# Menghapus kolom "FirstName" dan "LastName" karena sudah diwakili dengan kolom "Name"

df1.drop(['FirstName', 'LastName'], axis=1, inplace=True)

#### **3.2.2 Membuat Kolom/ Fitur baru "HandlingTime"**
"HandlingTime" merupakan selisih hari dari order diterima ("OrderDate") dengan barang dikirim ("ShippedDate"). Informasi ini menjelaskan seberapa cepat employee merespon permintaan pelanggan. Parameter ini merupakan salah satu indikator penting kinerja employee.

In [192]:
# Menambahkan Kolom "Handling Time" sebagai hasil operasi "ShippedDate" - "OrderDate"
df1['HandlingTime'] = df1['ShippedDate'] - df1['OrderDate']

#### **3.2.3 Membuat Kolom/ Fitur baru "OTD"**
"OTD" atau on time delivery merupakan selisih hari dari permintaan pelanggan barang untuk dikirim ("ReuiredDate") dengan barang dikirim ("ShippedDate"). Seperti "HandlingTime", parameter ini dapat menjadi tolak ukur dari kinerja employee karena berhubungan langsung dengan kepuasan pelanggan.

In [193]:
# Menambahkan kolom "OTD" atau on time delivery sebagai hasil operasi "ShippedDate" - "RequiredDate"
df1['OTD'] = df1['ShippedDate'] - df1['RequiredDate']

#### **3.2.4 Membuat Kolom/ Fitur baru "ProfitEach"**
Salah satu hal yang mendasar pada perusahaan profit tentu adalah profit itu sendiri. Profit dari setiap item penjualan dapat diperoleh dari pengurangan harga jual dengan harga beli.

In [194]:
# menambahkan kolom "ProfitEach" sebagai hasil operasi "UnitPriceSell" - "UnitPriceBuy"
df1['ProfitEach'] = df1['UnitPriceSell'] - df1['UnitPriceBuy']

### **3.3 MENGECEK DATA YANG TELAH DIBERSIHKAN**

#### **3.3.1 Menggunakan Method "sample"**

In [195]:
# Menampilakan sample 10 data teratas dari data yang sudah dibersihkan
df1.sample(10)

,EmployeeRegion,EmployeeTerritory,EmployeeID,Name,Title,TitleOfCourtesy,HireDate,Country,ReportsTo,Salary,...,ShipCountry,ProductID,UnitPriceBuy,Quantity,Discount,UnitPriceSell,CategoryName,HandlingTime,OTD,ProfitEach
9592,Northern ...,Portsmouth ...,9,Anne Dodsworth,Sales Representative,Ms.,1994-11-15,UK,5.0,2333.33,...,Germany,60,34.0,21,0.0,34.0,Dairy Products,NaT,NaT,0.0
1836,Eastern ...,Cambridge ...,2,Andrew Fuller,"Vice President, Sales",Dr.,1992-08-14,USA,NaN,2254.49,...,USA,57,19.5,15,0.0,19.5,Grains/Cereals,10 days,-18 days,0.0
2079,Eastern ...,Braintree ...,2,Andrew Fuller,"Vice President, Sales",Dr.,1992-08-14,USA,NaN,2254.49,...,Ireland,18,62.5,8,0.0,62.5,Seafood,3 days,-25 days,0.0
6899,Westerns ...,Chicago ...,7,Robert King,Sales Representative,Mr.,1994-01-02,UK,5.0,1991.55,...,Argentina,28,45.6,8,0.0,45.6,Produce,3 days,-11 days,0.0
6789,Westerns ...,Chicago ...,7,Robert King,Sales Representative,Mr.,1994-01-02,UK,5.0,1991.55,...,Italy,46,9.6,20,0.0,12.0,Seafood,7 days,-21 days,2.4
2583,Southern ...,Atlanta ...,3,Janet Leverling,Sales Representative,Ms.,1992-04-01,USA,2.0,3119.15,...,UK,31,12.5,50,0.0,12.5,Dairy Products,7 days,-21 days,0.0
7291,Westerns ...,Santa Monica ...,7,Robert King,Sales Representative,Mr.,1994-01-02,UK,5.0,1991.55,...,Mexico,17,31.2,8,0.0,39.0,Meat/Poultry,9 days,-19 days,7.8
4677,Eastern ...,Cary ...,4,Margaret Peacock,Sales Representative,Mrs.,1993-05-03,USA,2.0,1861.08,...,Mexico,40,18.4,10,0.0,18.4,Seafood,8 days,-20 days,0.0
963,Eastern ...,Bedford ...,2,Andrew Fuller,"Vice President, Sales",Dr.,1992-08-14,USA,NaN,2254.49,...,Brazil,63,35.1,16,0.0,43.9,Condiments,2 days,-26 days,8.8
6844,Westerns ...,Chicago ...,7,Robert King,Sales Representative,Mr.,1994-01-02,UK,5.0,1991.55,...,Austria,13,6.0,13,0.0,6.0,Seafood,3 days,-25 days,0.0


#### **3.3.2 Menggunakan Method "info"**

In [196]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10012 entries, 0 to 10128
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   EmployeeRegion     10012 non-null  object         
 1   EmployeeTerritory  10012 non-null  object         
 2   EmployeeID         10012 non-null  int64          
 3   Name               10012 non-null  object         
 4   Title              10012 non-null  object         
 5   TitleOfCourtesy    10012 non-null  object         
 6   HireDate           10012 non-null  datetime64[ns] 
 7   Country            10012 non-null  object         
 8   ReportsTo          8325 non-null   float64        
 9   Salary             10012 non-null  float64        
 10  OrderID            10012 non-null  int64          
 11  CustomerID         10012 non-null  object         
 12  OrderDate          10012 non-null  datetime64[ns] 
 13  RequiredDate       10012 non-null  datetime64[

#### **3.3.3 Menggunakan Method "describe"**

In [197]:
df1.describe()

,EmployeeID,ReportsTo,Salary,OrderID,ProductID,UnitPriceBuy,Quantity,Discount,UnitPriceSell,HandlingTime,OTD,ProfitEach
count,10012.000000,8325.000000,10012.000000,10012.000000,10012.000000,10012.000000,10012.00000,10012.0,10012.00000,9723,9723,10012.000000
mean,4.882841,3.206847,2251.006784,10664.161207,40.554934,26.554852,24.32521,0.0,28.21185,8 days 09:43:31.416229558,-20 days +08:22:39.580376427,1.656998
std,2.482696,1.471163,433.291513,240.557692,22.094110,30.691181,19.39474,0.0,32.44483,6 days 21:21:57.706722328,9 days 01:59:23.184784871,4.436528
min,1.000000,2.000000,1744.210000,10248.000000,1.000000,2.000000,1.00000,0.0,2.50000,1 days 00:00:00,-41 days +00:00:00,0.000000
25%,3.000000,2.000000,1991.550000,10459.750000,22.000000,12.000000,10.00000,0.0,12.50000,4 days 00:00:00,-24 days +00:00:00,0.000000
50%,5.000000,2.000000,2100.500000,10660.500000,41.000000,18.400000,20.00000,0.0,19.45000,7 days 00:00:00,-21 days +00:00:00,0.000000
75%,7.000000,5.000000,2333.330000,10869.000000,60.000000,32.800000,30.00000,0.0,34.00000,9 days 00:00:00,-18 days +00:00:00,1.950000
max,9.000000,5.000000,3119.150000,11077.000000,77.000000,263.500000,130.00000,0.0,263.50000,37 days 00:00:00,23 days 00:00:00,52.700000


#### **3.3.4 Membuat General Info Dari Dataframe**

In [198]:
# General Info Data Yang Telah Dibersihkan
list_item_akhir = []

for col in df1.columns:
    list_item_akhir.append([col, df1[col].dtype, len(df1), df1[col].isna().sum(), round((df1[col].isna().sum() / len(df1[col])) * 100, 2),
    df1[col].nunique(), list(df1[col].drop_duplicates().sample(1).values)])

df1_desc_akhir = pd.DataFrame(
    ListItem,
    columns= ['Column Name', 'Data Type', 'Data Count', 'Missing Value', 'Missing Value Percentage', 'Number of Unique', 'Unique Sample']
)

df1_desc_akhir

,Column Name,Data Type,Data Count,Missing Value,Missing Value Percentage,Number of Unique,Unique Sample
0,EmployeeRegion,object,10012,0,0.00,4,[Northern ...
1,EmployeeTerritory,object,10012,0,0.00,48,[Redmond ...
2,EmployeeID,int64,10012,0,0.00,9,[6]
3,Name,object,10012,0,0.00,9,[Laura Callahan]
4,Title,object,10012,0,0.00,4,[Inside Sales Coordinator]
5,TitleOfCourtesy,object,10012,0,0.00,4,[Mr.]
6,HireDate,datetime64[ns],10012,0,0.00,8,[1994-11-15T00:00:00.000000000]
7,Country,object,10012,0,0.00,2,[UK]
8,ReportsTo,float64,10012,1687,16.85,2,[nan]
9,Salary,float64,10012,0,0.00,9,[2254.49]


Dari informasi diatas, khususnya general info data, diketahui bahwa kolom "Discount" hanya memiliki 1 data unik yaitu 0. Artinya tidak ada diskon yang pernah diberikan. Oleh karena itu, kita akan hapus kolom "Discount" karena tidak ada data yang dapat dianalisa.

##### **3.3.4.1 Mengghapus Kolom "Discount**

In [199]:
# Menghapus kolom "Discount"

df1.drop(['Discount'], axis=1, inplace=True)

##### **3.3.4.2 Menggecek Kembali Method "info" Untuk Memastikan kolom "Discount" Terhapus**

In [200]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10012 entries, 0 to 10128
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   EmployeeRegion     10012 non-null  object         
 1   EmployeeTerritory  10012 non-null  object         
 2   EmployeeID         10012 non-null  int64          
 3   Name               10012 non-null  object         
 4   Title              10012 non-null  object         
 5   TitleOfCourtesy    10012 non-null  object         
 6   HireDate           10012 non-null  datetime64[ns] 
 7   Country            10012 non-null  object         
 8   ReportsTo          8325 non-null   float64        
 9   Salary             10012 non-null  float64        
 10  OrderID            10012 non-null  int64          
 11  CustomerID         10012 non-null  object         
 12  OrderDate          10012 non-null  datetime64[ns] 
 13  RequiredDate       10012 non-null  datetime64[

Dari tabel diatas, diketahui bahwa kolom "Discount" telah terhapus

## **4. ANALISIS DATA DAN VISUALISASI**

### **Analisa Profit Yang Dihasilkan per Employee**

In [203]:
# Membuat kolom/ fitur profit
df1['Profit'] = df1['ProfitEach'] * df1['Quantity']

df1['Profit'].describe()

count    10012.000000
mean        44.044978
std        165.614467
min          0.000000
25%          0.000000
50%          0.000000
75%         24.000000
max       2635.000000
Name: Profit, dtype: float64

In [204]:
# Melihat profit yang dihasilkan oleh tiap employee

df1[['Profit']].groupby(df1['EmployeeID']).sum()

,Profit
EmployeeID,
1,27148.16
2,53521.02
3,50216.68
4,74767.59
5,37068.60
6,27376.75
7,103062.00
8,42599.32
9,25218.20


Dalam pembahasan ini, Kita akan abaikan employee dengan ID 2, 5, 8 karena memiliki jabatan funsional selain sebagai sales dan berfokus pada 6 employee yang lain.

Dari tabel diatas, diketahui bahwa employee dengan ID 7, 4 dan 3 merupakan employee top performer dalam hal total profit yang dihasilkan. Sementara itu, employee dengan ID 9, 1, dan 6 merupakan employee yang under perform secara profit yang dihasilkan.

In [205]:
# melihat total transaksi yang dilakukan oleh tiap employee
df1[['OrderID']].groupby(df1['EmployeeID']).count()

,OrderID
EmployeeID,
1,690
2,1687
3,1284
4,1260
5,702
6,840
7,1760
8,1040
9,749


In [218]:
%%html
<div class='tableauPlaceholder' id='viz1649084994715' style='position: relative'><noscript><a href='#'><img alt='Total Profit dan Total Transaksi Yang Dihasilkan Tiap Employee ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;To&#47;TotalProfitYangDihasilkanTiapEmployee4&#47;Sheet1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='TotalProfitYangDihasilkanTiapEmployee4&#47;Sheet1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;To&#47;TotalProfitYangDihasilkanTiapEmployee4&#47;Sheet1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1649084994715');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

Secara total transaksi yang dilakukan, 3 employee yang top performer memiliki rata-rata transaksi yang lebih besar dibandingan dengan 3 employee yang under perform. Hal ini mengindikasikan bahwa, total profit sejalan dengan total transaksi yang dibuat.

In [206]:
# melihat performa employee dilihat dari presentase order yang belum/ tidak terselesaikan
round((df1['ShippedDate'].isna().groupby(df1['EmployeeID']).sum() / df1['OrderID'].groupby(df1['EmployeeID']).count() * 100), 2)

EmployeeID
1    8.99
2    3.73
3    0.00
4    2.62
5    0.00
6    2.38
7    2.84
8    3.85
9    2.80
dtype: float64

Sementara itu, presentase transaksi yang tidak terselesaikan oleh tiap employee, yaitu rasio antara total order yang tidak ada keterangan dikolom "ShippedDate" dengan total "OrderID" yang dihandle tiap employee ditampilkan ditabel diatas. Nampak bahwa tidak ada perbedaan yang signifikan antara employee yang top performer (ID 7, 4, dan 3) dan under perform (ID 9, 1, dan 6). Hal ini mengindikasikan bahwa parameter ini tidak bisa dijadikan acuan untuk menilai kinerja employee dan ada faktor lain yang lebih krusial dalam menentukan performance employee dalam menghasilkan profit.

### **Menyimpan Dataframe Dalam Excel Untuk Diolah di Tableau**

In [210]:
# simpan data dalam format excel
df1.to_excel('processed_northwind_employees.xlsx', index=False)

In [221]:
# Melihat data HandlingTime tiap employee

# Sebelum menentukan jenis data apa yang akan kita olah (rata-rata/ median) pertama kita tentukan dulu, apakah data terdistribusi normal atau tidak.

# import library
from scipy.stats import shapiro
listID = [1, 3, 4, 6, 7, 9]
# pengujian distribusi normal

for i in listID:
    stats, p_val = shapiro(df1['HandlingTime'][df1['EmployeeID'] == i])
    if p_val >= 0.05:
        print(f'Nilai p_val = {p_val}, "HandlingTime" EmployeeID {i} terdistribusi normal')
    else:
        print(f'Nilai p_val = {p_val}, "HandlingTime" EmployeeID {i} TIDAK terdistribusi normal')


Nilai p_val = 1.0, "HandlingTime" EmployeeID 1 terdistribusi normal
Nilai p_val = 8.589959586311129e-43, "HandlingTime" EmployeeID 3 TIDAK terdistribusi normal
Nilai p_val = 1.0, "HandlingTime" EmployeeID 4 terdistribusi normal
Nilai p_val = 1.0, "HandlingTime" EmployeeID 6 terdistribusi normal
Nilai p_val = 1.0, "HandlingTime" EmployeeID 7 terdistribusi normal
Nilai p_val = 1.0, "HandlingTime" EmployeeID 9 terdistribusi normal


C:\Users\sionrebeca\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_morestats.py:1758: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


Karena semua data terdistribusi normal, maka kita pilih mean/ rata-rata sebagai basis analisa "HandlingTime" dari tiap employee.

In [222]:
%%html
<div class='tableauPlaceholder' id='viz1649086360339' style='position: relative'><noscript><a href='#'><img alt='Rata-Rata &quot;HandlingTime&quot; Tiap Employee ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ra&#47;Rata-RataHandlingTimeTiapEmployee&#47;Sheet2&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Rata-RataHandlingTimeTiapEmployee&#47;Sheet2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ra&#47;Rata-RataHandlingTimeTiapEmployee&#47;Sheet2&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1649086360339');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

Dari tabel diatas, kita tahu bahwa "HandlingTime" Tidak memiliki korelasi dengan performa employee untuk menghasilkan profit. Employees top performer (ID 3, 4, dan 7) tidak memiliki nilai rata-rata "HandlingTime" yang lebih baik (lebih kecil) dari pada employee yang underperform (ID 1, 6, dan 9)

In [223]:
%%html
<div class='tableauPlaceholder' id='viz1649086982213' style='position: relative'><noscript><a href='#'><img alt='Tabel Atasan Dari Setiap Employee ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ta&#47;TabelAtasanDariSetiapEmployee&#47;Sheet3&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='TabelAtasanDariSetiapEmployee&#47;Sheet3' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ta&#47;TabelAtasanDariSetiapEmployee&#47;Sheet3&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1649086982213');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

Adakalanya, faktor kecakapan atasan menjadi penentu dari faktor dari pekerja untuk dapat perform secara optimal. Namun, dengan melihat tabel diatas diketahui bahwa hal yang demikian tidak menjadi penyebab utama seseorang pekerja dapat menghasilkan profit yang lebih besar. Employees yang top performer ada yang di bawah empoloyee dengan ID 2 dan juga 5. Hal yang sama juga terjadi untuk employee yang under perform.

In [224]:
%%html
<div class='tableauPlaceholder' id='viz1649087506338' style='position: relative'><noscript><a href='#'><img alt='Asal Negara Employee ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;As&#47;AsalNegaraEmployee&#47;Sheet4&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='AsalNegaraEmployee&#47;Sheet4' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;As&#47;AsalNegaraEmployee&#47;Sheet4&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1649087506338');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

In [225]:
%%html
<div class='tableauPlaceholder' id='viz1649087689010' style='position: relative'><noscript><a href='#'><img alt='Asal Negara Customer ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;As&#47;AsalNegaraCustomer&#47;Sheet4&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='AsalNegaraCustomer&#47;Sheet4' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;As&#47;AsalNegaraCustomer&#47;Sheet4&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1649087689010');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

Proximity atau kedekatan baik itu secara fisik atau psikis sering kali menjadi faktor penting dalam berbagai hal. Namun dalam kasus ini, proximity negara asal employee dan customer tidak menjadi penyebab utama seseorang pekerja dapat menghasilkan profit yang lebih besar. Employees yang top performer ada yang berasal dari UK (ID 7) dan sebaliknya employees yang underperform ada juga yang berasal dari USA (ID 1)yang mana menjadi pasar utama dari perusahaan.

In [226]:
%%html
<div class='tableauPlaceholder' id='viz1649088109491' style='position: relative'><noscript><a href='#'><img alt='Gaji dari Tiap Employee ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ga&#47;GajidariTiapEmployee&#47;Sheet5&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='GajidariTiapEmployee&#47;Sheet5' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ga&#47;GajidariTiapEmployee&#47;Sheet5&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1649088109491');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

Tidak pernah dipungkiri, salah satu yang menjadi motif dan juga motivasi utama seseorang adalah uang. Sebagaimana ditunjukkan pda gambar diatar, employees top performer (ID 3, 4, dan 7) memiliki gaji yang lebih besar dengan employees under perdorm (ID 1, 6, dan 9). Jadi, iming-iming kenaikan gaji dapat menjadi pemicu employee untuk bekerja lebih baik lagi sehingga meningkatkan profit perusahaan.`

## **5. KESIMPULAN**
#### **Faktor - faktor yang mempengaruhi kinerja employees dalam mengahasilkan profit bagi perusahaan adalah:**
- total transaksi yang dilakukan oleh tiap employee. Meningkatnya jumlah transaksi beriringan dengan meningkatnya nilai profit yang dihasilkan.
- salary yang diterima employee. Employee yang menerima gaji lebih tinggi cenderung menghasilkan total transaksi dan profit yang lebih tinggi untuk perusahaan.

#### **Rekomendasi ke perusahaan:**
- Mendorong employe untuk membuat transaksi sebanyak-banyaknya dengan mengiming-imingi kenaikan gaji guna meningkatkan total profit perusahaan.
